In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random
import cv2
from PIL import Image

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import LabelBinarizer



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from scipy.stats import skew
from scipy.sparse import csr_matrix


In [2]:
image_dir = "Datasets/VMMRdb/"

In [11]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2023-05-05 14:34:08--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2023-05-05 14:34:08 (6.85 MB/s) - ‘coco.names’ saved [625/625]



In [18]:
net = cv2.dnn.readNet("models/yolov4-tiny.weights", "models/yolov4-tiny.cfg")

# Load classes
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [20]:
classes[:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

In [21]:
input_size = (416, 416)
scale = 1/255.0

In [46]:
output_dir = "Datasets/yolov4/"

for root, dirs, files in os.walk('Datasets/VMMRdb/'):
    for filename in files:
        # Check if output file already exists
        output_path = os.path.join('Datasets/yolov4', os.path.basename(root), filename)
        if os.path.exists(output_path):
            continue  # skip file if it already exists

        # Load image
        img = cv2.imread(os.path.join(root, filename))
        if img is None:
            continue  # skip file if unable to load

        orig_h, orig_w = img.shape[:2]
        blob = cv2.dnn.blobFromImage(img, scale, input_size, swapRB=True, crop=False)

        # Set input and output layers
        net.setInput(blob)
        output_layers = net.getUnconnectedOutLayersNames()

        # Forward pass through network
        layer_outputs = net.forward(output_layers)

        # Process detections
        boxes = []
        confidences = []
        class_ids = []
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * orig_w)
                    center_y = int(detection[1] * orig_h)
                    width = int(detection[2] * orig_w)
                    height = int(detection[3] * orig_h)
                    x = int(center_x - width/2)
                    y = int(center_y - height/2)
                    boxes.append([x, y, width, height])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-max suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Draw final bounding boxes and class labels
        if len(indices) > 0:
            for i in indices.flatten():
                box = boxes[i]
                x, y, w, h = box
                label = classes[class_ids[i]]
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)
        cv2.imwrite(output_path, img)